<a href="https://colab.research.google.com/github/Teasotea/DialogSystem/blob/main/ConversationalAI_improved.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import libraries

In [ ]:
!pip install transformers

In [ ]:
!pip install nltk

In [ ]:
import numpy as np
import pandas as pd
import time
import os
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
import nltk
from nltk.stem.lancaster import LancasterStemmer
import requests
import re

In [ ]:
stemmer = LancasterStemmer()

# Part I: Greeting Classification

In [ ]:
training_data = []
greetings = ['hi', "hola", 'hey', 'hello','morning', 'evening', 'good day', 'good morning', 'greetings', 'howdy', 'welcome', 'bonjour',
             'buenas noches', 'buenos dias', 'salutation', 'salut', 'hail', 'salaam', 'aloha', 'ciao', 'good wishes', 'respects', 'high-five',
             'aloha', 'yoo-hoo', 'yawp', 'psst', 'oh', 'toast', 'ave', "how is it going?", 'yo', 'hi there']
# other = ['face','wisecrack','care','thick','reference','deserve','engine','cry','mud','worth',
#          'railroad','permanent','throne','tradition','loan','employ','resource','privilege','parachute',
#          'rent','of','characteristic','coin','teenager','established','reveal','bad','undress','revoke','ward']
for i in greetings:
  training_data.append({"class":"greeting", "sentence":i})
# for i in other:
#   training_data.append({"class":"other", "sentence":i})
training_data.append({"class":"other", "sentence":'word'})
  
greet_df = pd.DataFrame(training_data)
greet_df

In [ ]:
nltk.download('punkt')
corpus_words = {}
class_words = {}
classes = list(set([a['class'] for a in training_data]))
for c in classes:
    class_words[c] = []

In [ ]:
classes

In [ ]:
for data in training_data:
    for word in nltk.word_tokenize(data['sentence']):
        # ignore a some things
        if word not in ["?", "'s"]:
            # stem and lowercase each word
            stemmed_word = stemmer.stem(word.lower())
            # have we not seen this word already?
            if stemmed_word not in corpus_words:
                corpus_words[stemmed_word] = 1
            else:
                corpus_words[stemmed_word] += 1

            # add the word to our words in class list
            class_words[data['class']].extend([stemmed_word])

# we now have each stemmed word and the number of occurances of the word in our training corpus (the word's commonality)
print ("Corpus words and counts: %s \n" % corpus_words)
# also we have all words in each class
print ("Class words: %s" % class_words)

In [ ]:
# calculate a score for a given class
def calculate_class_score(sentence, class_name, show_details=True):
    score = 0
    # tokenize each word in our new sentence
    for word in nltk.word_tokenize(sentence):
        # check to see if the stem of the word is in any of our classes
        if stemmer.stem(word.lower()) in class_words[class_name]:
            # treat each word with same weight
            score += 1
            
            if show_details:
                print ("   match: %s" % stemmer.stem(word.lower() ))
    return score

In [ ]:
# we can now calculate a score for a new sentence
sentence = "good day for us to have lunch?"

# now we can find the class with the highest score
for c in class_words.keys():
    print ("Class: %s  Score: %s \n" % (c, calculate_class_score(sentence, c)))

In [ ]:
# calculate a score for a given class taking into account word commonality
def calculate_class_score_commonality(sentence, class_name, show_details=True):
    score = 0
    # tokenize each word in our new sentence
    for word in nltk.word_tokenize(sentence):
        # check to see if the stem of the word is in any of our classes
        if stemmer.stem(word.lower()) in class_words[class_name]:
            # treat each word with relative weight
            score += (1 / corpus_words[stemmer.stem(word.lower())])

            if show_details:
                print ("   match: %s (%s)" % (stemmer.stem(word.lower()), 1 / corpus_words[stemmer.stem(word.lower())]))
    return score

In [ ]:
# now we can find the class with the highest score
for c in class_words.keys():
    print ("Class: %s  Score: %s \n" % (c, calculate_class_score_commonality(sentence, c)))

In [ ]:
# return the class with highest score for sentence
def classify(sentence):
    high_class = 'other'
    high_score = 0
    # loop through our classes
    for c in class_words.keys():
        # calculate score of sentence for each class
        score = calculate_class_score_commonality(sentence, c, show_details=False)
        # keep track of highest score
        if score > high_score:
            high_class = c
            high_score = score

    return high_class

In [ ]:
classify("oh! are u a human?")

# Part II: Question Answering

In [ ]:
!pip install datasets

In [ ]:
import datasets

# [ds for ds in datasets.list_datasets() if 'ml' in ds.lower()]

In [ ]:
qa_ds = datasets.load_dataset('squad', streaming = False)
qa_ds

In [ ]:
qa_ds['train'].description

In [ ]:
print(qa_ds['train'].dataset_size)
qa_ds['train'].features

In [ ]:
qa_ds['train'].to_pandas().head()

In [ ]:
from transformers import BertTokenizer
b_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [ ]:
qa_ds['train'] = qa_ds['train'].map(
    lambda x: b_tokenizer(
        x['question'], x['context'], max_length = 512, padding = 'max_length', truncation = True
    ), batched = True, batch_size = 32
)

# QA Dataset 

In [ ]:
!pip install jupyterlab
!pip install python-Levenshtein
!pip install bert-serving-server bert-serving-client

In [ ]:
qa_data = pd.read_csv('https://raw.githubusercontent.com/Kizuna-Cheng/Data_Science_Interviews_NLP/main/data.csv')
qa_data.head(7)

In [ ]:
qa_data.Questions[:10].tolist()

In [ ]:
test_data = [
             'What does linear regression stands for?',
 'What is the differencebetween collinearity and multicollinearity?',
 'What are the cons of using a linear model?\n',
 'What are ridge and lasso regression?',
 'How does K-Nearest Neighbor work?',
 'How to select k for k means?',
 'Why is Naive Bayes “naive”?',
 'When should I use SVM?',
'What is pruning in decision trees?',
 'What are random forests? Why is Naive Bayes better?']


# QA Baseline

In [ ]:
def getResults(questions, fn):
    def getResult(q):
        answer, score, prediction = fn(q)
        return [q, prediction, answer, score]
    return pd.DataFrame(list(map(getResult, questions)), columns=["Q", "Prediction", "A", "Score"])
data

In [ ]:
def getNaiveAnswer(q):
    row = qa_data.loc[qa_data['Questions'].str.contains(re.sub(r"[^\w'\s)]+", "", q),case=False)]
    if len(row) > 0:
        return row["Answers"].values[0], 1, row["Questions"].values[0]
    else: return "Sorry, I didn't get you", 0, ""
print(getNaiveAnswer('How does K-Nearest Neighbor work?'))
getResults(test_data, getNaiveAnswer)

# DistilBERT model from HuggingFace

# Part III: Natural Language Generation

In [ ]:
# checkpoint 
checkpoint = "microsoft/DialoGPT-medium"
# download and cache tokenizer
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
# download and cache pre-trained model
modelNLG = AutoModelForCausalLM.from_pretrained(checkpoint)

# Part IV: Chatbot Development

In [ ]:
#change the code later, make it better
class ChatBot():
    def __init__(self):
        # once chat starts, the history will be stored for chat continuity
        self.chat_history_ids = None
        # make input ids global to use them anywhere within the object
        self.bot_input_ids = None
        # a flag to check whether to end the conversation
        self.end_chat = False
        # greet while starting
        self.welcome()
        self.is_greeting = False
        self.is_question_from_context = False
        self.answer = ''
        
    def welcome(self):
        print("Initializing ChatBot ...")
        # some time to get user ready
        time.sleep(2)
        print('Type "bye" or "quit" or "exit" to end chat \n')
        # give time to read what has been printed
        time.sleep(3)

        
    def user_input(self):
        # receive input from user
        text = input("User    >> ")
        # end conversation if user wishes so
        if text.lower().strip() in ['bye', 'quit', 'exit']:
            # turn flag on 
            self.end_chat=True
            # a closing comment
            print('ChatBot >>  See you soon! Bye!')
            time.sleep(1)
            print('\nQuitting ChatBot ...')
        else:
            # continue chat, preprocess input text
            # encode the new user input, add the eos_token and return a tensor in Pytorch
            if classify(text) == 'other':
              self.answer = getNaiveAnswer(text)[0]
              if self.answer != "Sorry, I didn't get you":
                self.is_question_from_context = True
              else: 
                self.new_user_input_ids = tokenizer.encode(text + tokenizer.eos_token, \
                                                       return_tensors='pt')
                self.answer = ''
            else: 
              self.is_greeting = True

    def bot_answer(self):
      print("ChatBot >>  " + self.answer)
      self.is_question_from_context = False
      self.answer = ''

    def bot_greet(self):
        greeting = np.random.choice([
            "Welcome, I am ChatBot, here for your kind service",
            "Hey, Great day! I am your virtual assistant",
            "Hello, it's my pleasure meeting you",
            "Hi, I am a ChatBot. Let's chat!"
        ])
        print("ChatBot >>  " + greeting)
        self.is_greeting = False

    def bot_response(self):
        # append the new user input tokens to the chat history
        # if chat has already begun
        if self.chat_history_ids is not None:
            self.bot_input_ids = torch.cat([self.chat_history_ids, self.new_user_input_ids], dim=-1) 
        else:
            # if first entry, initialize bot_input_ids
            self.bot_input_ids = self.new_user_input_ids
        
        # define the new chat_history_ids based on the preceding chats
        # generated a response while limiting the total chat history to 1000 tokens, 
        self.chat_history_ids = modelNLG.generate(self.bot_input_ids, max_length=1000, \
                                               pad_token_id=tokenizer.eos_token_id)
            
        # last ouput tokens from bot
        response = tokenizer.decode(self.chat_history_ids[:, self.bot_input_ids.shape[-1]:][0], \
                               skip_special_tokens=True)
        # in case, bot fails to answer
        if response == "":
            response = self.random_response()
        # print bot response
        print('ChatBot >>  '+ response)
        
    def random_response(self):
        i = -1
        response = tokenizer.decode(self.chat_history_ids[:, self.bot_input_ids.shape[i]:][0], \
                               skip_special_tokens=True)
        # iterate over history backwards to find the last token
        while response == '':
            i = i-1
            response = tokenizer.decode(self.chat_history_ids[:, self.bot_input_ids.shape[i]:][0], \
                               skip_special_tokens=True)
        # if it is a question, answer suitably
        if response.strip() == '?':
            reply = np.random.choice(["I don't know", 
                                     "I am not sure"])
        # not a question? answer suitably
        else:
            reply = np.random.choice(["Great", 
                                      "Fine. What's up?", 
                                      "Okay"
                                     ])
        return reply

In [ ]:
# build a ChatBot object
bot = ChatBot()
# start chatting
while True:
    # receive user input
    bot.user_input()
    # check whether to end chat
    if bot.end_chat:
        break
    # output bot response
    if bot.is_greeting == True:
       bot.bot_greet() 
    elif bot.is_question_from_context == True:
       bot.bot_answer()
    else: bot.bot_response()   

Initializing ChatBot ...
Type "bye" or "quit" or "exit" to end chat 

ChatBot >>  Hello, it's my pleasure meeting you
ChatBot >>  Hello, it's my pleasure meeting you
ChatBot >>  I would conduct an A/B test to determine if the introduction of a new feature results in a statistically significant improvement in a given metric that we care about. The metric(s) chosen depends on the goal of the feature. For example, a feature may be introduced to increase conversion rates, or web traffic, or retention rates.
First I would formulate my null hypothesis (feature X will not improve metric A) and my alternative hypothesis (feature X will improve metric A).
Next, I would create my control and test group through random sampling. Because the t-test inherently considers the sample size, I’m not going to specify a necessary sample size, although the larger the better.
Once I collect my data, depending on the characteristics of my data, I’d then conduct a t-test, Welch’s t-test, chi-squared test, or a